#### 0. LangChain에서 도구(tool) 활용 방법

In [ ]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:2])

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

In [ ]:
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(
#     base_url="https://api.groq.com/openai/v1",
#     model="meta-llama/llama-4-scout-17b-16e-instruct",
#     #model="moonshotai/kimi-k2-instruct-0905",
#     temperature=0
# )

from langchain_upstage import ChatUpstage
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

- [tool decorator](https://python.langchain.com/docs/how_to/custom_tools/#tool-decorator)를 사용하면 쉽게 도구를 만들 수 있습니다

In [ ]:
from langchain_core.tools import tool

@tool
def add(a: int, b: int) -> int:
    """숫자 a와 b를 더합니다."""
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """숫자 a와 b를 곱합니다."""
    return a * b

In [ ]:
# @tool 데코레이터로 선언된 함수는 직접 호출할 수 없다.
#add(10,20)

- LLM을 호출했을 때와 도구를 사용했을 때의 차이를 알아봅니다

In [ ]:
query = '3 곱하기 5는?'
llm_result = llm.invoke(query)

print(llm_result.content)

- 도구 리스트는 LLM에 해당하는 `BaseModel` 클래스에 `bind_tools` 메서드를 통해 전달합니다

In [ ]:
llm_with_tools = llm.bind_tools([add, multiply])

print(type(llm_with_tools))
print(llm_with_tools)

- `AIMessage`의 `additional_kwargs` 속성은 `tool_calls`를 포함합니다
- `tool_calls`는 도구를 호출하는 메시지를 포함합니다

In [ ]:
tool_result = llm_with_tools.invoke(query)

tool_result

In [ ]:
tool_result.tool_calls

In [ ]:
#query = "안녕하세요? 오늘의 날씨는 어떤가요?"
query = "10 더하기 5는 얼마인가요?"
tool_result = llm_with_tools.invoke(query)
print(tool_result.tool_calls)

In [ ]:
from typing import Sequence

from langchain_core.messages import AnyMessage, HumanMessage

query = "20과 10을 곱한 결과는?"
human_message = HumanMessage(query)
message_list: Sequence[AnyMessage] = [human_message] 

print(message_list)


- `tool_calls` 속성은 도구를 호출하는 메시지를 포함합니다
- `tool_calls`를 가진 `AIMessage`의 형태를 기억하기

In [ ]:
# llm_with_tools 는 LLM과 Tool이 연동된 RunnableBinding 객체
ai_message = llm_with_tools.invoke(message_list)
ai_message

In [ ]:
ai_message.tool_calls

In [ ]:
from pprint import pprint

message_list.append(ai_message)

pprint(message_list)

- `AIMessage`의 `tool_calls`를 활용해서 도구를 직접 호출할 수도 있습니다

In [ ]:
ai_message.tool_calls

In [ ]:
tool_message = multiply.invoke(ai_message.tool_calls[0])

print(tool_message)

- 하지만 에이전트의 경우 도구를 직접 호출하는 것이 아니라 도구를 호출하는 메시지를 만들어서 전달합니다

In [ ]:
message_list.append(tool_message)

pprint(message_list)

In [ ]:
tool_result = llm_with_tools.invoke(message_list)

pprint(tool_result)

- `message_list`의 순서를 기억하기

In [ ]:
message_list